# Best Model Load

In [5]:
#Set path = Path of model_64.h5 file
path = "/content/drive/MyDrive/Minor Project 3/Checkpoints/Success 2/model_64[1].h5"

In [6]:
#Loading and compiling the model
x =Input(shape=(frame, row, col))
encoded_rows = TimeDistributed(LSTM(row_hidden))(x) 
encoded_columns =LSTM(col_hidden)(encoded_rows)

layer2 = Dense(128, activation='relu')(encoded_columns)
layer3 = Dropout(.2)(layer2)
layer4 = Dense(64, activation='relu')(layer3)
layer5 = Dropout(.2)(layer4)
prediction = Dense(num_classes, activation='softmax')(layer5)

model = Model(x, prediction)
model.load_weights(path)

In [7]:
model.compile(loss='categorical_crossentropy', optimizer='NAdam', metrics=['accuracy']) 
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 99, 144, 256)]    0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 99, 128)           197120    
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense (Dense)                (None, 128)               16512     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0     

# Alert System

In [123]:
def send_alertt(flag, camera_number):
  
  print('Prediction Probabilities are\nNegative = ', flag[0][0],'\nPositive = ',flag[0][1])

  if(flag[0][1] > 0.6):
    print('Accident Detected\n')
    send_alert(camera_number)
  else:
    print('No accident')

In [127]:
#path = "/content/drive/MyDrive/Minor Project 3/Run Cases/Positive/"
path = "/content/drive/MyDrive/Minor Project 3/Run Cases/Negative/"
video_name = '104.mp4'

camera_number = 1

cap = cv2.VideoCapture(path+video_name)
property_id = int(cv2.CAP_PROP_FRAME_COUNT) 
length = int(cv2.VideoCapture.get(cap, property_id))
count = 0
success = 1

#print(length)

if length > 100:
  cut = length-99
else:
  cut = 0

#os.chdir("/content/drive/MyDrive/Minor Project 3/Run Cases/Frames/")
#dir =  path+'/'+video_name
#os.mkdir(video_name)
#os.chdir("/content/drive/MyDrive/Minor Project 3/Run Cases/Frames/"+video_name)

video_ready = []

for i in range(length):
  success, image = cap.read()
  #print(success)
  if count >= cut :
    n = count-cut
    #cv2.imwrite(str(n)+'.jpg', image)
    tmp = skimage.color.rgb2gray(np.array(image))
    tmp = transform.resize(tmp, (144, 256))
    video_ready.append(tmp)
  count += 1

#path = "/content/drive/MyDrive/Minor Project 3/Run Cases/Frames/"
#cur_video = glob.glob(path + '*.mp4') 

#video_ready = make_dataset(cur_video)

video_ready = [video_ready]
video_ready = np.array(video_ready)

video_ready.shape

flag = model.predict(video_ready)

print_accuracy(flag, camera_number)

Prediction Probabilities are
Negative =  0.52326816 
Positive =  0.47673184
No accident


In [122]:
import pandas as pd
import smtplib
import os

'''
Change these to your credentials and name

'''

def send_alert(camera_number):
  your_email = 'minorproject888@gmail.com'
  your_password = 'dssvouztedbhbrjt'

  # If you are using something other than gmail
  # then change the 'smtp.gmail.com' and 465 in the line below
  server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
  server.ehlo()
  server.login(your_email, your_password)

  # Read the file
  email_list = pd.read_excel("/content/drive/MyDrive/Minor Project 3/Email/EmailL.xlsx")
  camera = pd.read_excel("/content/drive/MyDrive/Minor Project 3/Email/Cameras.xlsx")

  gps_location = camera.loc[camera_number]

  # Get all the Names, Email Addreses, Subjects and Messages
  all_emails = email_list['Email']

  # Loop through the emails
  for idx in range(len(all_emails)):

      # Get each records name, email, subject and message
      email = all_emails[idx]
      subject = 'Requesting Immediate Help'
      message = 'Urgent - An accident has occurred.\nLocation -> ' + str(gps_location['Location']) + '\nGoogle Maps Link -> ' + str(gps_location['GPS Link'])
      # Create the email to send
      full_email = ("From: <{0}>\n"
                    "To:  <{1}>\n"
                    "Subject: {2}\n\n"
                    "{3}"
                    .format(your_email, email, subject, message))

      # In the email field, you can add multiple other emails if you want
      # all of them to receive the same text
      try:
          server.sendmail(your_email, [email], full_email)
          print('Email to {} successfully sent!\n'.format(email))
      except Exception as e:
          print('Email to {} could not be sent :( because {}\n\n'.format(email, str(e)))

  # Close the smtp server
  server.close()

In [111]:
camera_number = 1
camera = pd.read_excel("/content/drive/MyDrive/Minor Project 3/Email/Cameras.xlsx")

gps_location = camera.loc[camera_number]
gps_location

Camera Number                                         2
Location         Gopal Nagar, Model Town, Delhi, 110009
GPS Link          https://goo.gl/maps/NFPbfA7jwQxLZ5Ro6
Name: 1, dtype: object